In [35]:
import numpy as np 
import pandas as pd


import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from math import pi
import pandas as pd

from bokeh.io import show
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data
from bokeh.io import output_notebook
from datetime import datetime, date, time, timedelta
import calendar
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

In [2]:
#Cargamos el csv

events = pd.read_csv('../events.csv')

/home/jhanvato/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Analisis visita + checkout + conversion

In [3]:
# aca podemos analizar por que por la url podemos saber si e suna venta o ocmpra
#events['operacion']= events['url'].str.split('/').str[1]
#events['operacion'].value_counts()

In [107]:
def plot_barras(tabla,campo,titulo):


    q = tabla[campo].value_counts().reset_index()
    fruits = q['index']
    counts = q[campo]

    source = ColumnDataSource(data=dict(fruits=fruits, counts=counts))

    p = figure(x_range=fruits, plot_height=350, toolbar_location=None, title=titulo)
    p.vbar(x='fruits', top='counts', width=0.9, source=source, legend="fruits",
           line_color='white', fill_color=factor_cmap('fruits', palette=Spectral6, factors=fruits))

    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.y_range.end = counts.max()
    p.legend.orientation = "horizontal"
    p.legend.location = "top_right"
    output_notebook()
    show(p)

In [37]:
def agregar_columnas_fecha(tabla):
    formato1 = "%Y-%m-%d %H:%M:%S"
    dia = ("Lunes", "Martes", "Miercoles", "Jueves","Viernes", "Sabado","Domingo")
    mes = ("Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre")
    tabla['mes'] = tabla['timestamp'].apply(lambda x: mes[datetime.strptime(x,formato1).month - 1])
    tabla['dia'] = tabla['timestamp'].apply(lambda x: dia[datetime.isocalendar(datetime.strptime(x,formato1))[2] - 1])
    tabla['hora'] = tabla['timestamp'].apply(lambda x: datetime.strptime(x,formato1).hour)



In [75]:
def plot_dia_hora(a):
    agregar_columnas_fecha(a)
    dia_hora = a.groupby(['dia','hora'])['timestamp'].count().reset_index()
    dia = ("Lunes", "Martes", "Miercoles", "Jueves","Viernes", "Sabado","Domingo")
    mes = ("Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre")
    hora = ("0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23")   

    colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
    mapper = LinearColorMapper(palette=colors, low=dia_hora.timestamp.min(), high=dia_hora.timestamp.max())

    TOOLS = "save,box_zoom,reset"

    p = figure(title="Cantidad por dia y hora ",
               x_range=hora, y_range=dia,
               x_axis_location="above", plot_width=900, plot_height=500,
               tools=TOOLS, toolbar_location='below',
               tooltips=[('dia', '@dia'),('hora', '@hora'),('cantidad', '@timestamp')])


    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "12pt"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = pi / 3

    p.rect(x="hora", y="dia", width=1, height=1,
           source=dia_hora,
           fill_color={'field': 'timestamp', 'transform': mapper},
           line_color=None)

    color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="12pt",
                         ticker=BasicTicker(desired_num_ticks=30),
                       formatter=PrintfTickFormatter(format="%d"),
                       label_standoff=10, border_line_color=None, location=(0, 0))
    p.add_layout(color_bar, 'right')
    output_notebook()
    show(p)      # show the plot

In [76]:
def plot_mes_dia(a):
    agregar_columnas_fecha(a)
    mes_dia = a.groupby(['mes','dia'])['timestamp'].count().reset_index()
    dia = ("Lunes", "Martes", "Miercoles", "Jueves","Viernes", "Sabado","Domingo")
    mes = ("Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre")
    hora = ("0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23")   

    colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
    mapper = LinearColorMapper(palette=colors, low=mes_dia.timestamp.min(), high=mes_dia.timestamp.max())

    TOOLS = "save,box_zoom,reset"

    p = figure(title="Cantidad por mes y dia",
               x_range=mes, y_range=dia,
               x_axis_location="above", plot_width=900, plot_height=500,
               tools=TOOLS, toolbar_location='below',
               tooltips=[('mes', '@mes'),('dia', '@dia'),('cantidad', '@timestamp')])


    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "12pt"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = pi / 3

    p.rect(x="mes", y="dia", width=1, height=1,
           source=mes_dia,
           fill_color={'field': 'timestamp', 'transform': mapper},
           line_color=None)

    color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="12pt",
                         ticker=BasicTicker(desired_num_ticks=15),
                       formatter=PrintfTickFormatter(format="%d"),
                       label_standoff=10, border_line_color=None, location=(0, 0))
    p.add_layout(color_bar, 'right')
    output_notebook()
    show(p)      # show the plot

### Creo una funcion que me devuelva taban sin columnas vacias

In [42]:
# Esta funcion toma la tabla y borra las columnas que tiene todas NAN par asu mejor estudio
def borrar_columnas_vacias(tabla):
    for columna in tabla.columns:
        if(tabla[columna].count() == 0):
            tabla = tabla.drop([columna],axis = 1)
    return tabla
            

### Creo mis tablas que solo contengan registros por eventos y es borro las columnas vacias

In [8]:
#separo en tablas por eventos
viewed_product = events[events['event'] == 'viewed product']
brand_listing  = events[events['event'] == 'brand listing']
visited_site   = events[events['event'] == 'visited site']
ad_campaign_hit  = events[events['event'] == 'ad campaign hit']
searched_products  = events[events['event'] == 'searched products']
search_engine_hit  = events[events['event'] == 'search engine hit']
checkout  = events[events['event'] == 'checkout']
staticpage = events[events['event'] == 'staticpage']
conversion = events[events['event'] == 'conversion']
lead = events[events['event'] == 'lead']

In [9]:
viewed_product = borrar_columnas_vacias(viewed_product)
viewed_product.head(2)

,timestamp,event,person,sku,model,condition,storage,color
2,2018-05-31 23:38:09,viewed product,0004b0a2,2694,iPhone 5s,Bom,32GB,Cinza espacial
4,2018-05-29 13:29:25,viewed product,0006a21a,15338,Samsung Galaxy S8,Bom,64GB,Dourado


In [10]:
viewed_product.shape

(528931, 8)

In [11]:
brand_listing  = borrar_columnas_vacias(brand_listing)
brand_listing.head(2)

,timestamp,event,person,skus
12,2018-04-09 20:12:53,brand listing,000a54b2,"5075,12536,12520,12661,12506,12549,10449,9175,..."
14,2018-04-09 20:13:20,brand listing,000a54b2,"5075,12536,12520,12661,12506,12549,10449,9175,..."


In [12]:
brand_listing.shape

(98635, 4)

In [13]:
visited_site   = borrar_columnas_vacias(visited_site)
visited_site.head(2)

,timestamp,event,person,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
1,2018-05-31 23:38:05,visited site,0004b0a2,Paid,New,Camaragibe,Pernambuco,Brazil,Smartphone,360x640,Android 6,Chrome Mobile 39
5,2018-05-29 13:29:26,visited site,0006a21a,Paid,New,Rio de Janeiro,Rio de Janeiro,Brazil,Smartphone,360x640,Android 5.1.1,Android 5.1


In [14]:
visited_site.shape

(87378, 12)

In [15]:
lead = borrar_columnas_vacias(lead)
lead.head(2)

,timestamp,event,person,model
3248,2018-04-17 22:11:19,lead,01139919,Samsung Galaxy On 7
6636,2018-04-07 11:37:11,lead,01bca043,iPhone 6 Plus


In [16]:
lead.shape

(448, 4)

In [17]:
ad_campaign_hit  = borrar_columnas_vacias(ad_campaign_hit)
ad_campaign_hit.head(2)

,timestamp,event,person,url,campaign_source
0,2018-05-31 23:38:05,ad campaign hit,0004b0a2,/comprar/iphone/iphone-5s,criteo
6,2018-05-29 13:29:27,ad campaign hit,0006a21a,/comprar/samsung/galaxy-s8,criteo


In [18]:
ad_campaign_hit.shape

(82827, 5)

In [19]:
searched_products  = borrar_columnas_vacias(searched_products)
searched_products.head(2)

,timestamp,event,person,skus,search_term
157,2018-02-06 02:29:49,searched products,00204059,"2692,6819,823,2779,13864,2784,8135,6805,2773,2...",moto g 4
159,2018-02-06 02:32:41,searched products,00204059,"2692,6819,823,2779,13864,2784,8135,6805,2773,2...",moto g 4


In [20]:
searched_products.shape

(56073, 5)

In [21]:
search_engine_hit  = borrar_columnas_vacias(search_engine_hit)
search_engine_hit.head(2)

,timestamp,event,person,search_engine
10,2018-04-09 20:12:31,search engine hit,000a54b2,Google
17,2018-05-24 11:21:07,search engine hit,000a54b2,Google


In [22]:
search_engine_hit.shape

(50957, 4)

In [23]:
conversion = borrar_columnas_vacias(conversion)
conversion.head(2)

,timestamp,event,person,sku,model,condition,storage,color
220,2018-03-20 17:46:12,conversion,00204059,3084,Motorola Moto X2,Muito Bom,32GB,Couro Vintage
2282,2018-04-26 22:28:53,conversion,00c13dee,6650,Samsung Galaxy Core Plus Duos TV,Muito Bom,4GB,Branco


In [24]:
conversion.shape

(1172, 8)

In [25]:
staticpage = borrar_columnas_vacias(staticpage)
staticpage.head(2)

,timestamp,event,person,staticpage
254,2018-05-28 22:45:54,staticpage,0024ad28,how-to-buy
350,2018-05-30 01:06:09,staticpage,002932b1,trust-trocafone


In [26]:
staticpage.shape

(3598, 4)

In [27]:
checkout  = borrar_columnas_vacias(checkout)
checkout.head(2)

,timestamp,event,person,sku,model,condition,storage,color
3,2018-05-31 23:38:40,checkout,0004b0a2,2694,iPhone 5s,Bom,32GB,Cinza espacial
7,2018-05-29 13:29:35,checkout,0006a21a,15338,Samsung Galaxy S8,Bom,64GB,Dourado


In [28]:
checkout.shape


(33735, 8)

### Creo mi tabla maestro de personas con algunos campos

In [29]:
#obtengo algunos datos personales tomando las registros de evento visista q tienen dichos campos
datos_persona = visited_site.loc[:,['person','city','region','country','device_type','browser_version','channel']]\
                            .drop_duplicates(subset='person')\
                            .set_index('person')
datos_persona.head()

,city,region,country,device_type,browser_version,channel
person,,,,,,
0004b0a2,Camaragibe,Pernambuco,Brazil,Smartphone,Chrome Mobile 39,Paid
0006a21a,Rio de Janeiro,Rio de Janeiro,Brazil,Smartphone,Android 5.1,Paid
000a54b2,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,Chrome 65.0,Paid
00184bf9,Ribeirão Preto,Sao Paulo,Brazil,Computer,Chrome 65.0,Organic
0019c395,Unknown,Unknown,Brazil,Smartphone,Chrome Mobile 66.0,Paid


In [30]:
datos_persona.shape

(26899, 6)

### Tabla de checkout and conversion

In [31]:
conver = conversion.reset_index().drop_duplicates(subset = ['person','sku']).set_index(['person','sku'])
check = checkout.reset_index().drop_duplicates(subset= ['person','sku']).set_index(['person','sku'])

### Tabla maestro de modelos

In [32]:
#armo una tabla maestro de skus con modelo + condicion + storage + color 
modelos = events.loc [:,['sku','model','condition','storage','color']]\
                .dropna(subset=['sku','model'])
#modelos['sku'] = modelos['sku'].astype(float).astype(int)
modelos = modelos.drop_duplicates(subset = ['sku']).set_index('sku').sort_values('sku')
modelos.head()



,model,condition,storage,color
sku,,,,
71.0,iPhone 4G,Excelente,8GB,Preto
74.0,iPhone 4G,Muito Bom,8GB,Preto
80.0,iPhone 4G,Muito Bom,8GB,Branco
81.0,iPhone 4G,Excelente,8GB,Branco
87.0,iPhone 4G,Muito Bom,16GB,Preto


### Analisis de lso que retornaron y d elos que no lo hicieron

In [33]:
# De las personas que visitaron la pagina por primera vez (26898) solo volvieron a hacerlo (11131)
visitantes = visited_site
nuevos = visitantes.loc[visitantes['new_vs_returning']=='New',:]
nuevos = nuevos.set_index('person')
retornaron = visitantes.loc[visitantes['new_vs_returning']=='Returning',:].drop_duplicates(subset='person')
retornaron = retornaron.set_index('person').loc[:,\
                          ['timestamp','city','region','country','device_type',\
                           'operating_system_version','browser_version','channel']]
#retornaron = retornaron.drop_duplicates(subset = 'person')
print(retornaron.shape)
print(nuevos.shape)

#podemos ver que menos del 50% volvio despues de visitar vez pagina


(11131, 8)
(26898, 11)


In [150]:
plot_barras(nuevos,'channel','agrupados por canal')

/home/jhanvato/.local/lib/python2.7/site-packages/bokeh/models/mappers.py:82: UserWarning: Palette length does not match number of factors. 6    Unknown
Name: index, dtype: object will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))


Loading BokehJS ...

In [77]:
plot_mes_dia(nuevos)

Loading BokehJS ...

In [78]:
plot_mes_dia(retornaron)

Loading BokehJS ...

In [79]:
plot_dia_hora(nuevos)

Loading BokehJS ...

In [80]:
plot_dia_hora(retornaron)

Loading BokehJS ...

In [360]:
retornaron.head()

,timestamp,city,region,country,device_type,operating_system_version,browser_version,channel
person,,,,,,,,
000a54b2,2018-05-24 11:21:07,Rio de Janeiro,Rio de Janeiro,Brazil,Computer,Windows 10,Chrome 66.0,Paid
00184bf9,2018-05-04 18:47:03,Ribeirão Preto,Sao Paulo,Brazil,Computer,Windows 10,Chrome 66.0,Direct
001bb7eb,2018-06-08 12:15:33,Unknown,Unknown,Brazil,Computer,Windows 7,Firefox 52,Paid
00204059,2018-01-31 16:54:29,Unknown,Unknown,Unknown,Computer,Windows 7,Chrome 63.0,Social
0024ad28,2018-05-24 15:13:12,Santa Cruz do Sul,Rio Grande do Sul,Brazil,Computer,Windows 7,IE 11,Paid


In [112]:
noRetornaron = nuevos.join(retornaron,how = 'left',rsuffix=' r')
noRetornaron = noRetornaron.loc[noRetornaron['city r'].isnull() ,\
                          ['city','region','country','device_type',\
                           'operating_system_version','browser_version','channel']]
noRetornaron.head()
#Tengouna lista de las personas que que visitaron 1 vez y no volvieron a hacerlo

,city,region,country,device_type,operating_system_version,browser_version,channel
person,,,,,,,
0004b0a2,Camaragibe,Pernambuco,Brazil,Smartphone,Android 6,Chrome Mobile 39,Paid
0006a21a,Rio de Janeiro,Rio de Janeiro,Brazil,Smartphone,Android 5.1.1,Android 5.1,Paid
0019c395,Unknown,Unknown,Brazil,Smartphone,Android 6,Chrome Mobile 66.0,Paid
001f1653,Unknown,Unknown,Brazil,Smartphone,Android 6,Chrome Mobile 39,Paid
0024a82b,Unknown,Unknown,Brazil,Computer,Windows 7,Chrome 66.0,Paid


In [113]:
noRetornaron.shape

(15768, 7)

In [38]:
# Aca tengo la cantidad de visitantes diferentes por tipo de dispositivo
nuevos['device_type'].value_counts()

Smartphone    14305
Computer      12073
Tablet          478
Unknown          42
Name: device_type, dtype: int64

In [108]:
plot_barras(nuevos,'device_type','Usuarios nuevos agrupados por dispositivo')

Loading BokehJS ...

In [39]:
#aca tengo una lista de personas q visitaron 1 vez y no volvieron a hacerlo agrupado por dispositivo
retornaron['device_type'].value_counts()

Computer      5481
Smartphone    5452
Tablet         180
Unknown         18
Name: device_type, dtype: int64

In [109]:
plot_barras(retornaron,'device_type','Usuarios que retornaron agrupados por dispositivo')

Loading BokehJS ...

In [114]:
noRetornaron['device_type'].value_counts()

Smartphone    8846
Computer      6599
Tablet         298
Unknown         25
Name: device_type, dtype: int64

In [115]:
plot_barras(noRetornaron,'device_type','Usuarios que no retornaron agrupados por dispositivo')

Loading BokehJS ...

In [41]:
#visitantes que entraron por 1ra vez con que sistema operativo
nuevos['operating_system_version'].value_counts().head(10)

Windows 7        5966
Windows 10       4541
Android 7        3148
Android 6        2959
Android 6.0.1    2282
Android 5.1.1    1357
Android 7.1.1    1031
Windows 8.1       887
Android 5.0.2     480
iOS 11.3          466
Name: operating_system_version, dtype: int64

In [42]:
#visitantes que entraron por 1ra y no volvieorn a hacerlo
retornaron['operating_system_version'].value_counts().head(10)

Windows 7        2640
Windows 10       2147
Android 7        1299
Android 6.0.1    1014
Android 6         667
Android 5.1.1     525
Android 7.1.1     452
Windows 8.1       406
Android 5.0.2     202
iOS 11.3          190
Name: operating_system_version, dtype: int64

In [43]:
#visitantes que entraron por 1ra y no volvieorn a hacerlo
noRetornaron['operating_system_version'].value_counts().head(10)

Windows 7        3317
Windows 10       2409
Android 6        2283
Android 7        1868
Android 6.0.1    1239
Android 5.1.1     834
Android 7.1.1     577
Windows 8.1       484
iOS 11.3          297
Android 5.0.2     276
Name: operating_system_version, dtype: int64

In [44]:
#por que version de buscador entraron por primera vez los visitantes
nuevos['browser_version'].value_counts().head(10)

Chrome 66.0           7292
Chrome Mobile 66.0    5290
Chrome Mobile 39      1617
Chrome 67.0           1364
Chrome Mobile 67.0    1173
Chrome Mobile 65.0    1061
Chrome 65.0            964
Mobile Safari 11       726
Chrome Mobile 64.0     585
Chrome 64.0            427
Name: browser_version, dtype: int64

In [45]:
retornaron['browser_version'].value_counts().head(10)

Chrome 66.0           3138
Chrome Mobile 66.0    2250
Chrome 65.0            622
Chrome Mobile 65.0     618
Chrome 67.0            534
Chrome Mobile 67.0     441
Chrome Mobile 64.0     362
Chrome 64.0            294
Mobile Safari 11       283
Chrome Mobile 63.0     183
Name: browser_version, dtype: int64

In [46]:
#de los que no volvieron cuales son los buscadores que usaron
noRetornaron['browser_version'].value_counts().head(10)

Chrome 66.0             4377
Chrome Mobile 66.0      3243
Chrome Mobile 39        1614
Chrome 67.0             1023
Chrome Mobile 67.0       889
Mobile Safari 11         446
Chrome Mobile 65.0       306
Firefox 60               279
Samsung Internet 6.4     223
Chrome 65.0              175
Name: browser_version, dtype: int64

### De las personas que entraron a la etapa de checkout cuantos se convirtieron en una conversion (person,sku)

In [121]:
print(check.shape)
print(conver.shape)

(31286, 7)
(904, 7)


In [122]:
check = check[['timestamp','event','model','condition','storage','color']]
conver = conver[['timestamp','event','model','condition','storage','color']]

In [123]:
#personas que entraon al checkout y terminaron con una conversion par aun producto
checkAndConver = check.join(conver,how = 'inner',rsuffix= 'de conversion')
checkAndConver = checkAndConver[['timestamp','timestampde conversion','model','condition','storage','color']]
checkAndConver.head()

,,timestamp,timestampde conversion,model,condition,storage,color
person,sku,,,,,,
00c13dee,6650.0,2018-04-26 22:15:27,2018-04-26 22:28:53,Samsung Galaxy Core Plus Duos TV,Muito Bom,4GB,Branco
00fdbb4b,3348.0,2018-06-11 01:47:34,2018-06-10 14:37:50,Samsung Galaxy S6 Flat,Muito Bom,32GB,Branco
0146a9df,2694.0,2018-03-16 13:41:36,2018-03-16 13:50:25,iPhone 5s,Bom,32GB,Cinza espacial
01db2fe6,6357.0,2018-02-09 21:54:43,2018-02-09 22:07:07,Samsung Galaxy J5,Bom,16GB,Preto
03bdcd81,3301.0,2018-01-09 16:44:17,2018-01-09 16:47:50,Samsung Galaxy S6 Edge,Excelente,32GB,Dourado


In [124]:
# Esto es raro por q hay pesonas que hicieron conversiones sin entrar primero por un checkout
checkAndConver.shape

(749, 6)

In [126]:
# Para poder tener mas datos sobre esas personas la jineamos con mi tabla personas
checkAndConverConDatos = checkAndConver 
checkAndConverConDatos = checkAndConverConDatos.reset_index().set_index('person') 
checkAndConverConDatos = checkAndConverConDatos.join(datos_persona,how = 'inner')
checkAndConverConDatos.head()

,sku,timestamp,timestampde conversion,model,condition,storage,color,city,region,country,device_type,browser_version,channel
person,,,,,,,,,,,,,
00c13dee,6650,2018-04-26 22:15:27,2018-04-26 22:28:53,Samsung Galaxy Core Plus Duos TV,Muito Bom,4GB,Branco,Belém,Para,Brazil,Smartphone,Chrome Mobile 65.0,Paid
00fdbb4b,3348,2018-06-11 01:47:34,2018-06-10 14:37:50,Samsung Galaxy S6 Flat,Muito Bom,32GB,Branco,São Bernardo do Campo,Sao Paulo,Brazil,Computer,Chrome 66.0,Organic
0146a9df,2694,2018-03-16 13:41:36,2018-03-16 13:50:25,iPhone 5s,Bom,32GB,Cinza espacial,Fortaleza,Ceara,Brazil,Computer,Chrome 64.0,Paid
01db2fe6,6357,2018-02-09 21:54:43,2018-02-09 22:07:07,Samsung Galaxy J5,Bom,16GB,Preto,Jundiaí,Sao Paulo,Brazil,Smartphone,Chrome Mobile 64.0,Paid
03bdcd81,3301,2018-01-09 16:44:17,2018-01-09 16:47:50,Samsung Galaxy S6 Edge,Excelente,32GB,Dourado,São Paulo,Sao Paulo,Brazil,Computer,Edge 15.15063,Direct


In [127]:
# Disminuyo la cnatidad por que oobviamente mi tabla de datos personales esta armada desde la tabla visitas
checkAndConverConDatos.shape

(684, 13)

In [149]:
plot_barras(checkAndConverConDatos,'channel','Agrupados por condicion')

Loading BokehJS ...

In [143]:
plot_mes_dia(checkAndConverConDatos)

Loading BokehJS ...

In [144]:
plot_dia_hora(checkAndConverConDatos)

Loading BokehJS ...

In [131]:
checkAndConverConDatos['device_type'].value_counts()

Computer      388
Smartphone    294
Tablet          2
Name: device_type, dtype: int64

In [129]:
plot_barras(checkAndConverConDatos,'device_type','usuariosque se hicieorn chckouty conversion para un sku agrupados por dispositivo')

Loading BokehJS ...

In [135]:
#personas q entraron al checkout pero no terminaron con una conversion sin daots
checkAndNoConver = check.join(conver,how = 'left',rsuffix= 'de conversion')
checkAndNoConver = checkAndNoConver[['timestamp','model','condition','storage','color']]

checkAndNoConver.head()

timestamp                 model  condition  \
person   sku                                                             
0004b0a2 2694.0   2018-05-31 23:38:40             iPhone 5s        Bom   
0006a21a 15338.0  2018-05-29 13:29:35     Samsung Galaxy S8        Bom   
000a54b2 12660.0  2018-05-24 11:34:32  Motorola Moto Z Play        Bom   
00184bf9 6355.0   2018-04-06 05:14:58     Samsung Galaxy J5  Excelente   
         213.0    2018-05-04 18:50:21             iPhone 5c  Muito Bom   

                 storage           color  
person   sku                              
0004b0a2 2694.0     32GB  Cinza espacial  
0006a21a 15338.0    64GB         Dourado  
000a54b2 12660.0    32GB           Preto  
00184bf9 6355.0     16GB           Preto  
         213.0      16GB          Branco

In [136]:
checkAndNoConver.shape

(31286, 5)

In [138]:
checkAndNoConverConDatos = checkAndNoConver 
checkAndNoConverConDatos = checkAndNoConverConDatos.join(datos_persona,on = 'person',how = 'inner')
checkAndNoConverConDatos.head()

timestamp                 model  condition  \
         sku                                                             
0004b0a2 2694.0   2018-05-31 23:38:40             iPhone 5s        Bom   
0006a21a 15338.0  2018-05-29 13:29:35     Samsung Galaxy S8        Bom   
000a54b2 12660.0  2018-05-24 11:34:32  Motorola Moto Z Play        Bom   
00184bf9 6355.0   2018-04-06 05:14:58     Samsung Galaxy J5  Excelente   
         213.0    2018-05-04 18:50:21             iPhone 5c  Muito Bom   

                 storage           color            city          region  \
         sku                                                               
0004b0a2 2694.0     32GB  Cinza espacial      Camaragibe      Pernambuco   
0006a21a 15338.0    64GB         Dourado  Rio de Janeiro  Rio de Janeiro   
000a54b2 12660.0    32GB           Preto  Rio de Janeiro  Rio de Janeiro   
00184bf9 6355.0     16GB           Preto  Ribeirão Preto       Sao Paulo   
         213.0      16GB          Branco  Ribeirão Preto       Sao Paulo   

                 country device_type   browser_version  channel  
         sku                                                     
0004b0a2 2694.0   Brazil  Smartphone  Chrome Mobile 39     Paid  
0006a21a 15338.0  Brazil  Smartphone       Android 5.1     Paid  
000a54b2 12660.0  Brazil    Computer       Chrome 65.0     Paid  
00184bf9 6355.0   Brazil    Computer       Chrome 65.0  Organic  
         213.0    Brazil    Computer       Chrome 65.0  Organic

In [139]:
checkAndNoConverConDatos.shape

(30555, 11)

In [145]:
plot_mes_dia(checkAndNoConverConDatos)

Loading BokehJS ...

In [146]:
plot_dia_hora(checkAndNoConverConDatos)

Loading BokehJS ...

In [140]:
checkAndNoConverConDatos['device_type'].value_counts()

Smartphone    15937
Computer      14063
Tablet          511
Unknown          44
Name: device_type, dtype: int64

In [141]:
plot_barras(checkAndNoConverConDatos,'device_type','usuariosque se hicieorn chckout y no conversion para un sku agrupados por dispositivo')

Loading BokehJS ...

In [84]:
a =retornaron['device_type'].value_counts().reset_index()
a
device_type

,index,device_type
0,Computer,5481
1,Smartphone,5452
2,Tablet,180
3,Unknown,18


In [103]:
plot_barras(retornaron,'device_type','Usuarios que retornaron agrupados por dispositivo')

Loading BokehJS ...